In [1]:
import numpy as np
# Imports
import pandas as pd
import re
import math
import collections

In [2]:
# Load DB
df_inter = pd.read_pickle("../data/dataframes/inter.pkl")

In [3]:
# Visu DB
df_inter

,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,Z9,PY,WC,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG
6,J,"Kao, CH; Wang, CC; Wang, HY","Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",A neural-based predictive model of the compres...,COMPUTERS AND CONCRETE,Article,waste LCD concrete; compressive strength; back...,MECHANICAL-PROPERTIES; REGRESSION,The Taiwanese liquid crystal display (LCD) ind...,"[Kao, Chih-Han] Natl Quemoy Univ, Dept Civil E...",...,4.0,2017,"Computer Science, Interdisciplinary Applicatio...",10.12989/cac.2017.19.5.457,Computer Science; Construction & Building Tech...,WOS:000403196900003,"Taiwan, Taiwan, Taiwan",{AI},"{15.1, 12.5}","{SDG12, SDG15}"
28,C,"Nugra, H; Abad, A; Fuertes, W; Galarraga, F; A...","Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",A Low-cost IoT Application for the Urban Traff...,2016 IEEE/ACM 20TH INTERNATIONAL SYMPOSIUM ON ...,Proceedings Paper,Congestion of vehicular traffic; Wireless Sens...,INTERNET; SYSTEM; THINGS,Congestion of vehicular traffic in the cities ...,"[Nugra, Hugo; Abad, Alejandra; Fuertes, Walter...",...,6.0,2016,"Computer Science, Hardware & Architecture; Com...",10.1109/DS-RT.2016.24,Computer Science,WOS:000391589800021,"Ecuador, Ecuador, Ecuador, Ecuador, Ecuador, E...",{AI},{12.5},{SDG12}
90,J,"Peng, YQ; Xiao, YX; Fu, ZT; Dong, YH; Zheng, Y...","Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",Precision irrigation perspectives on the susta...,JOURNAL OF CLEANER PRODUCTION,Article,Field crop; Water demand prediction; Irrigatio...,REGRESSION-MODEL; DRIP IRRIGATION; NEURAL-NETW...,There is a global shortage of fresh water to m...,"[Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong,...",...,25.0,2019,Green & Sustainable Science & Technology; Engi...,10.1016/j.jclepro.2019.04.347,Science & Technology - Other Topics; Engineeri...,WOS:000474676300032,"China, China, China, China, China, China, Chin...",{AI},"{12.5, 12.1, 7.3, 2.4}","{SDG12, SDG2, SDG7}"
101,J,"Alvarez-de-los-Mozos, E; Renteria-Bilbao, A; D...","Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",WEEE Recycling and Circular Economy Assisted b...,APPLIED SCIENCES-BASEL,Article,human-robot collaboration; collaborative robot...,ELECTRONIC EQUIPMENT WEEE,Considering the amount of waste of electrical ...,"[Alvarez-de-los-Mozos, Esther; Diaz-Martin, Fe...",...,5.0,2020,"Chemistry, Multidisciplinary; Engineering, Mul...",10.3390/app10144800,Chemistry; Engineering; Materials Science; Phy...,WOS:000554798600001,"Spain, Spain, Spain",{robotics},{12.5},{SDG12}
218,J,"Laskurain-Iturbe, I; Arana-Landin, G; Landeta-...","Laskurain-Iturbe, Iker; Arana-Landin, German; ...",Exploring the influence of industry 4.0 techno...,JOURNAL OF CLEANER PRODUCTION,Article,Industry 4; 0; Circular economy; Literature re...,SMART PRODUCTION; SUSTAINABILITY; CHALLENGES; ...,"In the last decade, both Industry 4.0 technolo...","[Laskurain-Iturbe, Iker; Arana-Landin, German]...",...,18.0,2021,Green & Sustainable Science & Technology; Engi...,10.1016/j.jclepro.2021.128944,Science & Technology - Other Topics; Engineeri...,WOS:000704509900001,"Spain, Spain, Spain, Spain","{additive_manufacturing, big_data, AI, robotic...","{12.5, 7.3}","{SDG12, SDG7}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244072,J,"Strengers, Y; Nicholls, L","Strengers, Yolande; Nicholls, Larissa",Aesthetic pleasures and gendered tech-work in ...,MEDIA INTERNATIONAL AUSTRALIA,Article,aesthetics; digital housekeeping; gender; hous...,NaN,This article analyses visions of everyday life...,"[Strengers, Yolande; Nicholls, Larissa] RMIT U...",...,35.0,2018,Communication,10.1177/1329878X17737661,Communication,WOS:000419575300008,"Australia, Australia",{IOT},{5.4},{SDG5}
1244214,J,"Kan, MY; Sullivan, O; Gershuny, J","Kan, Man Yee; Sullivan, Oriel; Gershuny, Jonathan",Gender Convergence in Domestic Work: Discernin...,SOCIOLOGY-THE JOURNAL OF THE BRITISH SOCIOLOGI...,Article,domestic work; gender convergence; time use,WELFARE-STATE; TIME-USE; CONTEXT; EQU

In [4]:
# Visu Authors-EM
test = df_inter[["AF", "EM"]]
test

,AF,EM
6,"Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",ccw@gcloud.csu.edu.tw
28,"Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",henugra@espe.edu.ec; maabad2@espe.edu.ec; wmfu...
90,"Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",lxxcau@cau.edu.cn
101,"Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",esther.alvarez@deusto.es; arantxa.renteria@tec...
218,"Laskurain-Iturbe, Iker; Arana-Landin, German; ...",iker.laskurain@ehu.eus; g.arana@ehu.eus; benat...
...,...,...
1244072,"Strengers, Yolande; Nicholls, Larissa",yolande.strengers@rmit.edu.au
1244214,"Kan, Man Yee; Sullivan, Oriel; Gershuny, Jonathan",oriel.sullivan@sociology.ox.ac.uk
1244296,"Cornwell, Benjamin; Gershuny, Jonathan; Sulliv...",cornwell@cornell.edu; j.gershuny@ucl.ac.uk; o....
1244855,"Abdulcadir, Jasmine; Dewaele, Romain; Firmenic...",jasmine.abdulcadir@hcuge.ch


In [5]:
def match_email_name(lstname, lstemail) -> dict:
    dic_mail_name = dict()
    if type(lstemail) == float or type(lstname) == float:
        return dic_mail_name

    lst_name = set(lstname)
    lst_email = set(lstemail)

    for email in lst_email:
        email = email.lower()
        for full_name in lst_name:
            split_name = re.split(', ', full_name)
            try:
                lastname = split_name[0].lower()
            except ValueError:
                lastname = ""
            try:
                firstname = split_name[1].lower()
            except (ValueError, IndexError):
                firstname= " "
            # Variation of emails
            root = "@"+email.split("@")[1]
            lst_variations = [firstname+root,
                              firstname[0]+lastname+root,
                              firstname[0]+"."+lastname+root,
                              firstname+"."+lastname+root,
                              firstname+lastname+root,
                              firstname+lastname[0]+root,
                              lastname[0]+firstname+root,
                              firstname[0]+lastname[0]+root,
                              firstname+lastname[0]+root,
                              lastname+root,
                              lastname+firstname+root,
                              lastname+"."+firstname+root,
                              lastname+firstname[0]+root
                              ]
            for variation in lst_variations:
                if email == variation:
                    dic_mail_name[email] = full_name
                    continue
            if email in dic_mail_name:
                continue
            elif lastname.lower() in email:
                dic_mail_name[email] = full_name
            elif lastname[:4] in email and firstname[0] in email:
                dic_mail_name[email] = full_name
            # Two first names
            elif firstname in email:
                dic_mail_name[email] = full_name
        if email not in dic_mail_name:
            dic_mail_name[email] = "no_match"
    return dic_mail_name


In [6]:
%%time
df_inter = df_inter.assign(lst_AF=lambda x: x.AF.str.split("; "))
df_inter = df_inter.assign(lst_EM=lambda x: x.EM.str.split("; "))
df_inter['email'] = np.vectorize(match_email_name)(df_inter['lst_AF'], df_inter['lst_EM'])
df_inter

CPU times: user 1.44 s, sys: 27.9 ms, total: 1.47 s
Wall time: 1.47 s


,PT,AU,AF,TI,SO,DT,DE,ID,AB,C1,...,DI,SC,UT,CN,lst_DT,lst_Target,lst_SDG,lst_AF,lst_EM,email
6,J,"Kao, CH; Wang, CC; Wang, HY","Kao, Chih-Han; Wang, Chien-Chih; Wang, Her -Yung",A neural-based predictive model of the compres...,COMPUTERS AND CONCRETE,Article,waste LCD concrete; compressive strength; back...,MECHANICAL-PROPERTIES; REGRESSION,The Taiwanese liquid crystal display (LCD) ind...,"[Kao, Chih-Han] Natl Quemoy Univ, Dept Civil E...",...,10.12989/cac.2017.19.5.457,Computer Science; Construction & Building Tech...,WOS:000403196900003,"Taiwan, Taiwan, Taiwan",{AI},"{15.1, 12.5}","{SDG12, SDG15}","[Kao, Chih-Han, Wang, Chien-Chih, Wang, Her -Y...",[ccw@gcloud.csu.edu.tw],{'ccw@gcloud.csu.edu.tw': 'no_match'}
28,C,"Nugra, H; Abad, A; Fuertes, W; Galarraga, F; A...","Nugra, Hugo; Abad, Alejandra; Fuertes, Walter;...",A Low-cost IoT Application for the Urban Traff...,2016 IEEE/ACM 20TH INTERNATIONAL SYMPOSIUM ON ...,Proceedings Paper,Congestion of vehicular traffic; Wireless Sens...,INTERNET; SYSTEM; THINGS,Congestion of vehicular traffic in the cities ...,"[Nugra, Hugo; Abad, Alejandra; Fuertes, Walter...",...,10.1109/DS-RT.2016.24,Computer Science,WOS:000391589800021,"Ecuador, Ecuador, Ecuador, Ecuador, Ecuador, E...",{AI},{12.5},{SDG12},"[Nugra, Hugo, Abad, Alejandra, Fuertes, Walter...","[henugra@espe.edu.ec, maabad2@espe.edu.ec, wmf...","{'ttoulkeridis@espe.edu.ec': 'Toulkeridis, The..."
90,J,"Peng, YQ; Xiao, YX; Fu, ZT; Dong, YH; Zheng, Y...","Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong, ...",Precision irrigation perspectives on the susta...,JOURNAL OF CLEANER PRODUCTION,Article,Field crop; Water demand prediction; Irrigatio...,REGRESSION-MODEL; DRIP IRRIGATION; NEURAL-NETW...,There is a global shortage of fresh water to m...,"[Peng, Yaoqi; Xiao, Yingxin; Fu, Zetian; Dong,...",...,10.1016/j.jclepro.2019.04.347,Science & Technology - Other Topics; Engineeri...,WOS:000474676300032,"China, China, China, China, China, China, Chin...",{AI},"{12.5, 12.1, 7.3, 2.4}","{SDG12, SDG2, SDG7}","[Peng, Yaoqi, Xiao, Yingxin, Fu, Zetian, Dong,...",[lxxcau@cau.edu.cn],{'lxxcau@cau.edu.cn': 'no_match'}
101,J,"Alvarez-de-los-Mozos, E; Renteria-Bilbao, A; D...","Alvarez-de-los-Mozos, Esther; Renteria-Bilbao,...",WEEE Recycling and Circular Economy Assisted b...,APPLIED SCIENCES-BASEL,Article,human-robot collaboration; collaborative robot...,ELECTRONIC EQUIPMENT WEEE,Considering the amount of waste of electrical ...,"[Alvarez-de-los-Mozos, Esther; Diaz-Martin, Fe...",...,10.3390/app10144800,Chemistry; Engineering; Materials Science; Phy...,WOS:000554798600001,"Spain, Spain, Spain",{robotics},{12.5},{SDG12},"[Alvarez-de-los-Mozos, Esther, Renteria-Bilbao...","[esther.alvarez@deusto.es, arantxa.renteria@te...","{'fernando.diaz@deusto.es': 'Diaz-Martin, Fern..."
218,J,"Laskurain-Iturbe, I; Arana-Landin, G; Landeta-...","Laskurain-Iturbe, Iker; Arana-Landin, German; ...",Exploring the influence of industry 4.0 techno...,JOURNAL OF CLEANER PRODUCTION,Article,Industry 4; 0; Circular economy; Literature re...,SMART PRODUCTION; SUSTAINABILITY; CHALLENGES; ...,"In the last decade, both Industry 4.0 technolo...","[Laskurain-Iturbe, Iker; Arana-Landin, German]...",...,10.1016/j.jclepro.2021.128944,Science & Technology - Other Topics; Engineeri...,WOS:000704509900001,"Spain, Spain, Spain, Spain","{additive_manufacturing, big_data, AI, robotic...","{12.5, 7.3}","{SDG12, SDG7}","[Laskurain-Iturbe, Iker, Arana-Landin, German,...","[iker.laskurain@ehu.eus, g.arana@ehu.eus, bena...","{'benat.landeta@ehu.eus': 'Landeta-Manzano, Be..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1244072,J,"Strengers, Y; Nicholls, L","Strengers, Yolande; Nicholls, Larissa",Aesthetic pleasures and gendered tech-work in ...,MEDIA INTERNATIONAL AUSTRALIA,Article,aesthetics; digital housekeeping; gender; hous...,NaN,This article analyses visions of everyday life...,"[Strengers, Yolande; Nicholls, Larissa] RMIT U...",...,10.1177/1329878X

In [7]:
test = df_inter[['lst_AF', "email"]]
test

,lst_AF,email
6,"[Kao, Chih-Han, Wang, Chien-Chih, Wang, Her -Y...",{'ccw@gcloud.csu.edu.tw': 'no_match'}
28,"[Nugra, Hugo, Abad, Alejandra, Fuertes, Walter...","{'ttoulkeridis@espe.edu.ec': 'Toulkeridis, The..."
90,"[Peng, Yaoqi, Xiao, Yingxin, Fu, Zetian, Dong,...",{'lxxcau@cau.edu.cn': 'no_match'}
101,"[Alvarez-de-los-Mozos, Esther, Renteria-Bilbao...","{'fernando.diaz@deusto.es': 'Diaz-Martin, Fern..."
218,"[Laskurain-Iturbe, Iker, Arana-Landin, German,...","{'benat.landeta@ehu.eus': 'Landeta-Manzano, Be..."
...,...,...
1244072,"[Strengers, Yolande, Nicholls, Larissa]","{'yolande.strengers@rmit.edu.au': 'Strengers, ..."
1244214,"[Kan, Man Yee, Sullivan, Oriel, Gershuny, Jona...",{'oriel.sullivan@sociology.ox.ac.uk': 'Sulliva...
1244296,"[Cornwell, Benjamin, Gershuny, Jonathan, Sulli...","{'cornwell@cornell.edu': 'Cornwell, Benjamin',..."
1244855,"[Abdulcadir, Jasmine, Dewaele, Romain, Firmeni...","{'jasmine.abdulcadir@hcuge.ch': 'Abdulcadir, J..."


In [8]:
tot_email = 0
tot_name= 0
tot_no_match = 0
tot_len_one_match = 0
test_email = test.loc[:, "email"]
for dic_email in test_email:
    if len(dic_email) == 1 and next(iter(dic_email.items()))[1] == "no match":
        tot_len_one_match += 1
        continue
    for email, name in dic_email.items():
        if name == "no_match":
            tot_no_match += 1
        else:
            tot_name += 1
        tot_email += 1
print(f"total email: {tot_email}\ntotal name: {tot_name}\ntotal no match: {tot_no_match}\nlen1nomatch: {tot_len_one_match}")

total email: 126777
total name: 102920
total no match: 23857
len1nomatch: 0


In [9]:
lst_no_match = []
for row in test.itertuples(name=None, index=False):
    lst_AF = row[0]
    email = row[1]
    for mail, name in email.items():
        if name == "no_match":

            lst_no_match.append([mail, name, "; ".join(lst_AF)])


In [12]:
df = pd.DataFrame(lst_no_match, columns= ['email', 'name', 'lst_names'])
df.to_excel("../data/explore_no_match.xlsx")

In [23]:

a = {'drazen@infosys.tuwien.ac.at': 'Lucanin, Drazen'}
next(iter(a.items()))[1]

'Lucanin, Drazen'

In [31]:
%%time
idx_sdg = {name:ind for ind, name in enumerate(df_sdg.columns)}
dic_authors = dict()
c = 0
for row in df_sdg.itertuples(index=False, name=None):
    if type(row[idx_sdg['WC']]) == float:
        wc = ["no_wc"]
    else:
        wc = row[idx_sdg['WC']].split("; ")
    for email, name in row[idx_sdg['email']].items():
        if email not in dic_authors:
            dic_authors[email] = {"name":name,
                                  "total_publications":1,
                                  "lst_publications": [row[idx_sdg['TI']]],
                                  "lst_date_pubs": [row[idx_sdg['PY']]],
                                  "dic_WC" : collections.Counter(wc),
                                  "total_citations": int(row[idx_sdg['TC']])
                                  }
        else:
            dic_authors[email]['total_publications'] += 1
            dic_authors[email]['lst_publications'].append(row[idx_sdg['TI']])
            dic_authors[email]['lst_date_pubs'].append(row[idx_sdg['PY']])
            dic_authors[email]['dic_WC'].update(collections.Counter(wc))
            dic_authors[email]['total_citations'] += int(row[idx_sdg['TC']])


CPU times: user 13.7 s, sys: 255 ms, total: 14 s
Wall time: 14 s


In [15]:
len(dic_authors)

1448354

In [16]:
more_than_one_pubs = {author:tot for author, tot in dic_authors.items() if tot > 1}

In [18]:
len(more_than_one_pubs)

389009

In [24]:
str_wc = 'Business; Economics'
c = collections.Counter(str_wc.split("; "))
c

Counter({'Business': 1, 'Economics': 1})

In [25]:
c.update(collections.Counter(["Stuff", "Business"]))
c

Counter({'Business': 2, 'Economics': 1, 'Stuff': 1})

In [32]:
dic_authors_show = {em:spec for em, spec in dic_authors.items() if spec['total_publications'] > 5}

In [33]:
dic_authors_show

{'mirela.matei@upg-ploiesti.ro': {'name': 'Panait, Mirela',
  'total_publications': 22,
  'lst_publications': ['Corporate Social Responsibility in the Telecommunication Industry-Driver of Entrepreneurship',
   'Understanding the Impact of Generation Z on Risk Management-A Preliminary Views on Values, Competencies, and Ethics of the Generation Z in Public Administration',
   'Financing Growth through Remittances and Foreign Direct Investment: Evidences from Balkan Countries',
   'Sustainable Tourism Issues in European Countries during the Global Pandemic Crisis',
   'Internationalization of Large Companies from Central and Eastern Europe or the Birth of New Stars',
   'APPROACHES REGARDING ENVIRONMENTAL KUZNETS CURVE IN THE EUROPEAN UNION FROM THE PERSPECTIVE OF SUSTAINABLE DEVELOPMENT',
   'Energy Poverty in European Union: Assessment Difficulties, Effects on the Quality of Life, Mitigation Measures. Some Evidences from Romania',
   'Is energy efficiency a robust driver for the new nor